# シングルタスクとマルチタスク学習の比較

このノートブックでは、T5モデルを使用してシングルタスク学習とマルチタスク学習を比較します。データセットとしてAMCDとMARCを使用します。


In [ ]:
import torch
import sys
import warnings

warnings.filterwarnings("ignore")
sys.path.append("/home/sakulab/workspace/nakasone/ex/multi_task_learning_with_t5")

from src.t5_model import T5Model
from src.dataset import SingleTaskDataset, MultiTaskDataset
import src.config as config

## シングルタスク学習


### AMCD

In [ ]:
# データセットのロード
train_dataset_AMCD = SingleTaskDataset(task_type="AMCD",split="train")
valid_dataset_AMCD = SingleTaskDataset(task_type="AMCD", split="valid")

# モデルの初期化
model_AMCD = T5Model(multi_task=False, experiment_name="ex_single_amcd")

# 訓練
model_AMCD.train(train_dataset_AMCD, valid_dataset_AMCD)

# モデルの保存
model_AMCD.save_model(config.MODEL_DIR + "/T5/t5_single_task/AMCD")

In [ ]:
# 評価
test_dataset_AMCD = SingleTaskDataset(task_type="AMCD", split="test")
AMCD_single_task_results = model_AMCD.evaluate(test_dataset_AMCD)
print("Single Task Results of AMCD:\n", AMCD_single_task_results)

### MARC

In [ ]:
# データセットのロード
train_dataset_MARC = SingleTaskDataset(task_type="MARC",split="train", data_limit=5600)
valid_dataset_MARC = SingleTaskDataset(task_type="MARC",split="valid",data_limit=465)

# モデルの初期化
model_MARC = T5Model(multi_task=False,  experiment_name="ex_single_marc")

# 訓練
model_MARC.train(train_dataset_MARC, valid_dataset_MARC)

# モデルの保存
# model_MARC.save_model(config.MODEL_DIR + "/T5/t5_single_task/MARC")

In [ ]:
# 評価
test_dataset_MARC = SingleTaskDataset(task_type="MARC", split="test", data_limit=934)
MARC_single_task_results = model_MARC.evaluate(test_dataset_MARC)
print("Single Task Results of MARC:\n", MARC_single_task_results)

### labelをencodeした場合
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}

In [ ]:
# データセットのロード
train_dataset_MARC_enc = SingleTaskDataset(task_type="MARC",split="train", data_limit=5600, label_encode=True)
valid_dataset_MARC_enc = SingleTaskDataset(task_type="MARC",split="valid",data_limit=465, label_encode=True)

# モデルの初期化
model_MARC_enc = T5Model(multi_task=False,  experiment_name="ex_single_marc")

# 訓練
model_MARC_enc.train(train_dataset_MARC_enc, valid_dataset_MARC_enc)

# モデルの保存
# model_MARC_enc.save_model(config.MODEL_DIR + "/T5/t5_single_task/MARC_enc")

In [ ]:
# 評価
test_dataset_MARC_enc = SingleTaskDataset(task_type="MARC", split="test", data_limit=934, label_encode=True)
MARC_enc_single_task_results = model_MARC_enc.evaluate(test_dataset_MARC_enc)
print("Single Task Results of MARC:\n", MARC_enc_single_task_results)

## マルチタスク学習

task_weights = {"MARC": 0.7, "AMCD": 0.3}

In [ ]:
# 訓練データと検証データのデータセットを作成し、2つのタスク（MARCとAMCD）のデータセットを組み合わせる
train_dataset_MARC = MultiTaskDataset(task_type="MARC", split="train", data_limit=5600)
train_dataset_AMCD = MultiTaskDataset(task_type="AMCD", split="train")
train_dataset = torch.utils.data.ConcatDataset([train_dataset_MARC, train_dataset_AMCD])

valid_dataset_MARC = MultiTaskDataset(task_type="MARC", split="valid", data_limit=465)
valid_dataset_AMCD = MultiTaskDataset(task_type="AMCD", split="valid")
valid_dataset = torch.utils.data.ConcatDataset([valid_dataset_MARC, valid_dataset_AMCD])

# 訓練
model = T5Model(multi_task=True,  experiment_name="ex_multi_07_03")
model.train(train_dataset, valid_dataset)
# model_AMCD.save_model(config.MODEL_DIR + "/T5/t5_multi_task")

In [ ]:
# AMCDの評価
test_dataset_AMCD = MultiTaskDataset(task_type="AMCD", split="test")
AMCD_results = model.evaluate(test_dataset_AMCD)
print("Results of AMCD:\n", AMCD_results)

In [ ]:
# MARCの評価
test_dataset_MARC = MultiTaskDataset(task_type="MARC", split="test")
MARC_results = model.evaluate(test_dataset_MARC)
print("Results of MARC:\n", MARC_results)

## TensorBoardによる

In [ ]:
# TensorBoardのインポート
%load_ext tensorboard

In [ ]:
# ログディレクトリの指定（SummaryWriterで指定したパスに合わせて変更）
log_dir = 'runs'
%tensorboard --logdir {log_dir}